# 分词

中文分词模型，提供目前比较常见的分词模型，结合代码进行实现。主要有词典(也叫机械分词)、N_gram、隐马尔可夫HMM、条件随机场CRF几种模型的分词。

# 数据


  ● 数据集来自SIGHAN，SIGHAN 是国际计算语言协会ACL中文处理小组的简称。目前SIGHAN bakeoff 已经举办了 6 届，其中语料资源免费。选用 icwb-data2 数据作为数据集。
  
  ● icwb-data2 中包含train、test、scripts、gold、doc 目录
  
      ○ doc：数据集的一些使用指南
      ○ training： 包含已经分词的训练数据集目录。这里选择 msr_training.utf8 作为训练集。其他信息可见doc目录下的说明
          ■ 文件后缀名为 utf8 的表示编码格式为 UTF-8。
          ■ 文件前缀 msr_ ，代表是微软亚洲研究院提供。
      ○ testing：未切分的测试数据集
      ○ scripts：评分脚本和简单的分词器
      ○ gold：测试数据集的标准分词和训练集中抽取的词表
      
  ● 数据集下载：http://sighan.cs.uchicago.edu/bakeoff2005/

## 词典分词

关于分词的介绍参考个人云笔记：http://note.youdao.com/noteshare?id=81e18f06e7c39d59da74323cc5aff346

词典分词模型已封装在 module/segmentation/dict/dict_segmentation.py 类中。详细代码在可在此文件中查看。模型提供以下函数

 1. 拟合：根据提供的训练数据创建词典。 并进行保存
 2. 评估：读取测试数据，并对数据进行分词，写入文本
 3. 分词：输入某个句子文本，对其进行分词
 4. 添加新词：对与某个句子中的词汇没有正确识别，可通过添加新词方式实现。
 5. 加载模型：对拟合保存的模型进行加载

### 加载库

In [2]:
# 加载库
import sys
sys.path.append("../")

from module.segmentation.dict.dict_segmentation import DictSegmentation

### 初始化
    def __init__(self, max_matching=MAX_MATCHING):
        """
        初始化对象
        :param max_matching: <int> 词的最大匹配长度。用于在正向匹配和逆向匹配中对词进行划分
        """

In [3]:
# 定义词的最大匹配长度。用于在正向匹配和逆向匹配中对词进行划分。
max_matching = 10 

# 初始化对象
dict_seg = DictSegmentation(max_matching=max_matching)

### 拟合
    def fit(self, file, split_lab,
            save_model="DictSegmentation.pickle",
            del_start_str=None,
            del_end_str=None,
            regular_func=None,
            ignore_punctuation=True):
        """
        拟合词典
        :param file: <str> 训练文件
        :param split_lab: <str> 训练文本中对词的划分标记
        :param save_model: <str> 保存模型的文件名
        :param del_start_str: <str> 对于训练数据中的文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: <str> 对于训练数据中的文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: <func> 正则化函数
        :param ignore_punctuation: <bool> 是否忽略训练数据中的标点符号，即是否将标点符号加入到词典中
        :return:
        """

In [4]:
# 语料数据的训练文件
train_file = '../data/icwb2-data/training/msr_training.utf8'
save_model = '../model/dict_segmentation.pickle'

dict_seg.fit(train_file, split_lab=' ', save_model=save_model)

86924it [00:00, 219482.90it/s]
100%|██████████| 86924/86924 [40:07<00:00, 20.28it/s]  


save dictionary success! File: ../model/dict_segmentation.pickle
word count： 88041


### 评估
    def eval(self, file, seg_lab=' ', w_file="test.txt", encoding="utf-8", threads=3, del_start_str=None,
             del_end_str=None, regular_func=None):
        """
        评估。采用线程并非机制，读取测试数据集，并进行分词
        :param file: <str> 测试数据文本
        :param seg_lab: <str> 分词标记。在分词完成之后，需要根据某个字符标记词汇，默认为 " "
        :param w_file: <str> 分词结果写入的文件名
        :param encoding: <str> 分词结果写入文件的编码格式
        :param threads: <int> 启动线程数量
        :param del_start_str: <str> 对于数据中的文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: <str> 对于数据中的文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: <func> 正则化函数
        :return:
        """

In [6]:
test_file = '../data/icwb2-data/testing/msr_test.utf8'
result = '../result/dict/msr_test_seg_result.utf8'

dict_seg.eval(test_file, seg_lab="  ", w_file=result, threads=5)

3985it [00:00, 148943.60it/s]


thread_0 start...
thread_1 start...
thread_2 start...
thread_3 start...
thread_4 start...
thread_0 Process:797/797 	thread_1 Process:797/797 	thread_2 Process:797/797 	thread_3 Process:797/797 	thread_4 Process:797/797 	Total process: 3985/3985 Percentage:100.00%

over！File: ../result/dict/msr_test_seg_result.utf8, encoding: utf-8


#### 评分

使用下载数据集中提供的 scripts 评分脚本对测试数据集进行评分。详细查看 README .  下面摘自README 。

* Scoring

The script 'score' is used to generate compare two segmentations. The
script takes three arguments:

1. The training set word list
2. The gold standard segmentation
3. The segmented test file

You must not mix character encodings when invoking the scoring
script. For example:

% perl scripts/score gold/cityu_training_words.utf8 gold/cityu_test_gold.utf8 test_segmentation.utf8 > score.ut8

In [7]:
!perl ../data/icwb2-data/scripts/score ../data/icwb2-data/gold/msr_training_words.utf8 ../data/icwb2-data/gold/msr_test_gold.utf8 ../result/dict/msr_test_seg_result.utf8 > ../result/dict/score.utf8

查看评分结果。评分结果输入到 ..／result/dict_seg_score.utf8 文件中. 结果在最后几行中。

In [8]:
!tail -22 ../result/dict/score.utf8

INSERTIONS:	0
DELETIONS:	4
SUBSTITUTIONS:	4
NCHANGE:	8
NTRUTH:	45
NTEST:	41
TRUE WORDS RECALL:	0.822
TEST WORDS PRECISION:	0.902
=== SUMMARY:
=== TOTAL INSERTIONS:	5095
=== TOTAL DELETIONS:	376
=== TOTAL SUBSTITUTIONS:	4194
=== TOTAL NCHANGE:	9665
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	111592
=== TOTAL TRUE WORDS RECALL:	0.957
=== TOTAL TEST WORDS PRECISION:	0.917
=== F MEASURE:	0.937
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.025
=== IV Recall Rate:	0.983
###	../result/dict/msr_test_seg_result.utf8	5095	376	4194	9665	106873	111592	0.957	0.917	0.937	0.026	0.025	0.983


### 分词

    def cut(self, text, seg_lab=' ', del_start_str=None, del_end_str=None, regular_func=None):
        """
        输入某个句子或文本，对其进行分词
        :param text: <str> 句子文本
        :param seg_lab: <str> 分词完成之后需要通过某个字符对其进行标记，默认为 " "
        :param del_start_str: <str> 对于文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: <str> 对于文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: <func> 正则化函数
        :return: <str> 分词文本
        """
        

In [9]:
sentence = "我爱吃雪糕。"
dict_seg.cut(sentence, seg_lab="/")

'我/爱/吃/雪/糕/。'

### 增加新词
    def add_word(self, words, is_save=True, model_file='DictSegmentation.pickle'):
        """
        添加词汇到训练词典中
        :param words: <str, list, tuple> 词汇，可以是字符、列表、元祖
        :param is_save: <bool> 对于新加的词汇，是否保存词典
        :param model_file: <str> 需要保存的模型文件名
        :return:
        """

In [10]:
dict_seg.add_word(["雪糕"], is_save=True, model_file=save_model)
dict_seg.cut(sentence, seg_lab="/")

save dictionary success! File: ../model/dict_segmentation.pickle


'我/爱/吃/雪糕/。'

### 加载模型
    def load(model_file, max_matching=MAX_MATCHING):
        """
        加载词典分词对象
        :param model_file: <str> 保存的词典文件，也叫模型文件
        :param max_matching: <int> 最大匹配词长度，用于正向和逆向匹配的最大匹配词长度
        :return: <DictSegmentation> 词典分词对象
        """

In [11]:
d_seg = dict_seg.load(save_model, max_matching=max_matching)

print(d_seg.cut("扬帆远东做与中国合作的先行", seg_lab="/"))
print(d_seg.cut("我爱吃雪糕。", seg_lab="/"))

扬帆/远东/做/与/中国/合作/的/先行
我/爱/吃/雪糕/。


## N-gram 分词



ngram分词模型已封装在 module/segmentation/ngram/ngram_segmentation.py 类中。详细代码在可在此文件中查看。模型提供以下函数

 1. 拟合：根据提供的训练数据创建词典。 并进行保存
 2. 评估：读取测试数据，并对数据进行分词，写入文本
 3. 分词：输入某个句子文本，对其进行分词
 4. 加载模型：对拟合保存的模型进行加载

### 加载库

In [1]:
import sys

sys.path.append("../")
from module.segmentation.ngram.ngram_segmentation import NgramSegmentation

### 初始化

    def __init__(self, n=NGRAM, max_matching=MAX_MATCHING):
        """
        初始化
        :param n: <int> ngram 中的n元计算模式。n = {2,3}
        :param max_matching: <int> 最大匹配长度默认=10
        """
 
 备注：n = {2, 3} n = 3 的时候需要足够大的内存，否则容易出现内存错误

In [2]:
n = 2
max_matching = 10

ngram_seg = NgramSegmentation(n, max_matching)

### 拟合

    def fit(self, file, split_lab, save_model="ngram_segmentation.pickle", smoothing=None, del_start_str=None,
            del_end_str=None, regular_func=None, is_save=False):
        """
        拟合。输入训练数据集。创建词典、统计词频
        :param file: <str> 文本数据
        :param split_lab: <str> 训练数据集中的分词标记
        :param save_model: <str> 保存模型
        :param smoothing: <str> n 阶词频概率计算模式。目前只有 Laplace 模式
        :param del_start_str: <str> 需要删除的开始字符
        :param del_end_str: <str> 需要删除的结束字符
        :param regular_func: <func> 正则化函数
        :param is_save: <bool> 是否保存训练的词典。这里自测保存词典容易出现内存错误。等待优化处理
        :return:
        """

In [4]:
train_file = "../data/icwb2-data/training/msr_training.utf8"
# save_model = "../model/ngram_segmentation.pickle"

ngram_seg.fit(train_file, split_lab="  ")

86924it [00:00, 214035.73it/s]
100%|██████████| 86918/86918 [00:05<00:00, 14791.20it/s]


word number:  88184 total words number: 2542224


### 评估

    def eval(self, file, seg_lab="  ", w_file="test.txt", encoding="utf-8", threads=3, del_start_str=None,
             del_end_str=None, regular_func=None):
        """
        评估。输入测试数据。对测试数据进行分词。
        :param file: <str> 测试数据文件名
        :param seg_lab: <str> 分词标记。默认"  "， 即在分词完成之后，使用该标记区分
        :param w_file: <str> 将结果写入的文件名
        :param encoding: <str> 写入文件的编码格式，默认为utf-8
        :param threads: <int> 线程数量
        :param del_start_str: <str> 需要删除的开始字符
        :param del_end_str: <str> 需要删除的结束字符
        :param regular_func: <func> 正则化函数
        :return:
        """

In [5]:
test_file = "../data/icwb2-data/testing/msr_test.utf8"
result = "../result/ngram/msr_test_seg_result.utf8"

ngram_seg.eval(test_file, seg_lab="  ", w_file=result, threads=5)

3985it [00:00, 132908.45it/s]


thread_0 start...
thread_1 start...thread_2 start...

thread_3 start...
thread_4 start...
thread_0 Process:797/797 	thread_1 Process:797/797 	thread_2 Process:797/797 	thread_3 Process:797/797 	thread_4 Process:797/797 	Total process: 3985/3985 Percentage:100.00%

over！File: ../result/ngram/msr_test_seg_result.utf8, encoding: utf-8


#### 评分

使用下载数据集中提供的 scripts 评分脚本对测试数据集进行评分。（同上）

In [6]:
!perl ../data/icwb2-data/scripts/score ../data/icwb2-data/gold/msr_training_words.utf8 ../data/icwb2-data/gold/msr_test_gold.utf8 ../result/ngram/msr_test_seg_result.utf8 > ../result/ngram/score.utf8

In [7]:
!tail -22 ../result/ngram/score.utf8

INSERTIONS:	0
DELETIONS:	4
SUBSTITUTIONS:	4
NCHANGE:	8
NTRUTH:	45
NTEST:	41
TRUE WORDS RECALL:	0.822
TEST WORDS PRECISION:	0.902
=== SUMMARY:
=== TOTAL INSERTIONS:	4954
=== TOTAL DELETIONS:	389
=== TOTAL SUBSTITUTIONS:	3567
=== TOTAL NCHANGE:	8910
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	111438
=== TOTAL TRUE WORDS RECALL:	0.963
=== TOTAL TEST WORDS PRECISION:	0.924
=== F MEASURE:	0.943
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.025
=== IV Recall Rate:	0.988
###	../result/ngram/msr_test_seg_result.utf8	4954	389	3567	8910	106873	111438	0.963	0.924	0.943	0.026	0.025	0.988


### 分词

    def cut(self, text, seg_lab=' ', del_start_str=None, del_end_str=None, regular_func=None):
        """
        分词。对某个文本或句子进行分词。
        :param text: <str> 文本
        :param seg_lab: <str> 分词标记。默认为 " " 。在分词完成之后，使用该标记标记完成的分词
        :param del_start_str: <str> 需要删除的开始字符
        :param del_end_str: <str> 需要删除的结束字符
        :param regular_func: <func> 正则化函数
        :return: <str> 分词结果
        """

In [8]:
ngram_seg.cut("扬帆远东做与中国合作的先行", seg_lab="/")

'扬帆/远东/做/与/中国/合作/的/先行'

## HMM(隐马尔可夫) 分词

hmm分词模型已封装在 module/segmentation/hmm/hmm_segmentation.py 类中。详细代码在可在此文件中查看。模型提供以下函数

1. fit 拟合模型。输入训练数据集。创建发射概率矩阵和转移概率矩阵
2. eval 评估。输入测试数据集。对测试数据进行分词，将分词结果写入文本
3. cut 分词。输入文本，对文本进行分词
4. load 加载模型。加载训练好的模型s

### 加载库

In [2]:
import sys
sys.path.append("../")

from module.segmentation.hmm.hmm_segmentation import HmmSegmentation

### 初始化

    def __init__(self):
        """
        初始化
        """

In [3]:
hmm_seg = HmmSegmentation()

### 拟合

    def fit(self, file, split_lab, save_model="hmm_segmentation.pickle", del_start_str=None, del_end_str=None,
            regular_func=None):
        """
        拟合
        :param file: <str> 训练文件数据
        :param split_lab: <str> 训练集中的分词标签
        :param save_model: <str> 训练完成之后保存模型的文件名
        :param del_start_str: <str>  对于文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: <str>  对于文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: <func> 正则化函数
        :return:
        """

In [4]:
train_file = "../data/icwb2-data/training/msr_training.utf8"
save_model = "../model/hmm_segmentation.pickle"

hmm_seg.fit(train_file, split_lab="  ", save_model=save_model, del_start_str="“")

86924it [00:00, 209800.76it/s]
100%|██████████| 5168/5168 [00:00<00:00, 854367.71it/s]
86924it [00:04, 20313.92it/s]
100%|██████████| 86924/86924 [00:04<00:00, 19643.09it/s]


word number:  5168
save dictionary success! File: ../model/hmm_segmentation.pickle


### 评估

    def eval(self, file, seg_lab="  ", w_file="test.txt", encoding="utf-8", threads=3, del_start_str=None,
             del_end_str=None, regular_func=None):
        """
        评估。采用线程并非机制，读取测试数据集，并进行分词
        :param file: <str> 测试数据文本
        :param seg_lab: <str> 分词标记。在分词完成之后，需要根据某个字符标记词汇，默认为 " "
        :param w_file: <str> 分词结果写入的文件名
        :param encoding: <str> 分词结果写入文件的编码格式
        :param threads: <int> 启动线程数量,默认为 3 
        :param del_start_str: <str> 对于数据中的文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: <str> 对于数据中的文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: <func> 正则化函数
        :return:
        """

In [5]:
test_file = "../data/icwb2-data/testing/msr_test.utf8"
result = "../result/hmm/msr_test_seg_result.utf8"

import re
def regular(sent):
    # 删除文本中可能存在的空格字符
    sent = re.sub("[ ]+", '', sent)
    return sent

hmm_seg.eval(test_file, seg_lab="  ", w_file=result, regular_func=regular)

3985it [00:00, 97043.58it/s]


thread_0 start...thread_1 start...

thread_2 start...
thread_0 Process:1328/1328 	thread_1 Process:1328/1328 	thread_2 Process:1329/1329 	Total process: 3985/3985 Percentage:100.00%

over！File: ../result/hmm/msr_test_seg_result.utf8, encoding: utf-8


#### 评分

使用下载数据集中提供的 scripts 评分脚本对测试数据集进行评分。（同上）

In [6]:
!perl ../data/icwb2-data/scripts/score ../data/icwb2-data/gold/msr_training_words.utf8 ../data/icwb2-data/gold/msr_test_gold.utf8 ../result/hmm/msr_test_seg_result.utf8 > ../result/hmm/score.utf8

In [7]:
!tail -22 ../result/hmm/score.utf8

INSERTIONS:	2
DELETIONS:	3
SUBSTITUTIONS:	5
NCHANGE:	10
NTRUTH:	45
NTEST:	44
TRUE WORDS RECALL:	0.822
TEST WORDS PRECISION:	0.841
=== SUMMARY:
=== TOTAL INSERTIONS:	7304
=== TOTAL DELETIONS:	5671
=== TOTAL SUBSTITUTIONS:	17396
=== TOTAL NCHANGE:	30371
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	108506
=== TOTAL TRUE WORDS RECALL:	0.784
=== TOTAL TEST WORDS PRECISION:	0.772
=== F MEASURE:	0.778
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.363
=== IV Recall Rate:	0.796
###	../result/hmm/msr_test_seg_result.utf8	7304	5671	17396	30371	106873	108506	0.784	0.772	0.778	0.026	0.363	0.796


### 分词

    def cut(self, text, seg_lab=' ', del_start_str=None, del_end_str=None, regular_func=None):
        """
        分词。
        :param text: <str> 文本。例如"小明是中国人"
        :param seg_lab: <str> 分词标记，分词完成之后使用该标记，进行区分。默认问 ' '
        :param del_start_str: <str>  对于文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: <str>  对于文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: <func> 正则化函数
        :return: <str> 分词文本
        """

In [8]:
hmm_seg.cut("扬帆远东做与中国合作的先行", seg_lab="/")

'扬帆/远东/做/与/中国/合作/的/先行'

### 加载模型

    def load(model_file):
        """
        加载模型
        :param model_file: <str> 模型文件
        :return: <HmmSegmentation> 分词模型
        """

In [9]:
save_model = "../model/hmm_segmentation.pickle"

h_mm = HmmSegmentation.load(save_model)
h_mm.cut("扬帆远东做与中国合作的先行", seg_lab="/")

'扬帆/远东/做/与/中国/合作/的/先行'

## CRF 分词

  ● CRF 也叫条件随机场，解决了在 HMM 分词问题上不能做特征选择的问题。同时采用全局归一化，解决了最大熵隐马尔可夫模型出现的标注偏置的问题。
  
  ● CRF 优缺点：
  
      ○ 优点：文字词语出现的频率信息，同时考虑上下文语境，具备较好的学习能力，因此其对歧义词和未登录词的识别都具有良好的效果
      ○ 缺点：训练周期较长，运营时计算量较大，性能不如词典分词
  
  ● 相关开源实现：
  
      ○ CRF++。  目前普遍认为比较好的分词工具包。但是目前没有可调用的API，只能根据提供的脚步使用。
          ■ 相关中文分词参考：51nlp 比较详细的说明了CRF++分词的操作说明。
          ■ CRF++ 文档
      ○ Genius。https://github.com/duanhongyi/genius
      ○ sklean_crfsuite.CRF: https://sklearn-crfsuite.readthedocs.io/en/latest/api.html

模型使用 [sklearn_crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/api.html) 进行构建。详细参考 module/segmentation/crf/crf_segmentation.py

### 加载库

In [1]:
import sys
sys.path.append("../")

from module.segmentation.crf.crf_segmentation import CRFSegmentation

### 初始化

    def __init__(self, algorithm='lbfgs', min_freq=0, c1=0, c2=1.0, max_iterations=None):
        """
        选用 sklearn_crfsuite AIP 文档中常用参数。详细的参数信息，可以参考 sklearn_crfsuite API 文档。
        sklearn_crfsuite API 文档：https://sklearn-crfsuite.readthedocs.io/en/latest/api.html
        """

In [2]:
crf = CRFSegmentation()

### 拟合

    def fit(self, file, split_lab, save_model="crf_segmentation.pickle", del_start_str=None, del_end_str=None,
            regular_func=None):
        """
        拟合模型
        :param file: (str, mandatory) 训练数据
        :param split_lab: (str, mandatory) 训练文本中对词的划分标记
        :param save_model: (str, optional, default='crf_segmentation.pickle') 保存模型的文件名
        :param del_start_str: (str, optional, default=None) 对于训练数据中的文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: (str, optional, default=None) 对于训练数据中的文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: (fun, optional, default=None)> 正则化函数
        :return:
        """

In [3]:
train_file = "../data/icwb2-data/training/msr_training.utf8"
save_model = "../model/crf_segmentation.pickle"

crf.fit(train_file, split_lab=" ", save_model=save_model, del_start_str="“")

86924it [00:00, 208464.88it/s]
100%|██████████| 86924/86924 [00:07<00:00, 11977.09it/s]


Save model over! File: ../model/crf_segmentation.pickle


### 评估

    def eval(self, file, seg_lab="  ", w_file="test.txt", encoding="utf-8", threads=3, del_start_str=None,
             del_end_str=None, regular_func=None):
        """
        评估。
        :param file: (str, mandatory) 测试数据
        :param seg_lab: (str, mandatory, default="  ") 分词完成之后使用该标记区分
        :param w_file: (str, optional, default="test.txt") 将分词结果写入文件
        :param encoding: (str, optional, default="utf-8") 写入文件的编码格式
        :param threads: (int optional, default=3) 执行线程数量
        :param del_start_str: (str, optional, default=None) 对于训练数据中的文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: (str, optional, default=None) 对于训练数据中的文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: (fun, optional, default=None)> 正则化函数
        :return:
        """

In [4]:
test_file = "../data/icwb2-data/testing/msr_test.utf8"
w_file = "../result/crf/msr_test_seg_result.utf8"

import re
def regular(sent):
    # 删除文本中可能存在的空格字符
    sent = re.sub("[ ]+", '', sent)
    return sent

crf.eval(test_file, seg_lab="  ", w_file=w_file, regular_func=regular)

3985it [00:00, 80155.67it/s]


thread_0 start...
thread_1 start...
thread_2 start...
thread_0 Process:1328/1328 	thread_1 Process:1328/1328 	thread_2 Process:1329/1329 	Total process: 3985/3985 Percentage:100.00%

over！File: ../result/crf/msr_test_seg_result.utf8, encoding: utf-8


#### 评分

使用下载数据集中提供的 scripts 评分脚本对测试数据集进行评分。（同上）

In [5]:
!perl ../data/icwb2-data/scripts/score ../data/icwb2-data/gold/msr_training_words.utf8 ../data/icwb2-data/gold/msr_test_gold.utf8 ../result/crf/msr_test_seg_result.utf8 > ../result/crf/score.utf8

In [6]:
!tail -22 ../result/crf/score.utf8

INSERTIONS:	2
DELETIONS:	1
SUBSTITUTIONS:	8
NCHANGE:	11
NTRUTH:	45
NTEST:	46
TRUE WORDS RECALL:	0.800
TEST WORDS PRECISION:	0.783
=== SUMMARY:
=== TOTAL INSERTIONS:	4506
=== TOTAL DELETIONS:	4801
=== TOTAL SUBSTITUTIONS:	11275
=== TOTAL NCHANGE:	20582
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	106578
=== TOTAL TRUE WORDS RECALL:	0.850
=== TOTAL TEST WORDS PRECISION:	0.852
=== F MEASURE:	0.851
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.587
=== IV Recall Rate:	0.857
###	../result/crf/msr_test_seg_result.utf8	4506	4801	11275	20582	106873	106578	0.850	0.852	0.851	0.026	0.587	0.857


### 分词

    def cut(self, text, seg_lab=' ', del_start_str=None, del_end_str=None, regular_func=None):
        """
        分词。
        :param text: (str, mandatory) 字符文本或句子
        :param seg_lab: (str, optional, default="  ") 分词完成之后使用该标记区分
        :param del_start_str: (str, optional, default=None) 对于训练数据中的文本句子，是否存在开始标记需要删除，如果有，则输入
        :param del_end_str: (str, optional, default=None) 对于训练数据中的文本句子，是否存在结束标记需要删除，如果有，则输入
        :param regular_func: (fun, optional, default=None)> 正则化函数
        :return: (str) 分词句子
        """

In [7]:
sent = "种田要有个明白账，投本要赚利润是起码的道理。"

crf.cut(sent, seg_lab="/")

'种田/要/有/个明/白账/，/投本/要/赚/利润/是/起码/的/道理/。'

### 加载模型

    def load(file):
        """
        加载模型。
        :param file: (str, mandatory) 模型文件
        :return: (MFSSegmentation) 分词对象
        """

In [8]:
model = "../model/crf_segmentation.pickle"

crf_model = CRFSegmentation.load(model)
crf_model.cut(sent, seg_lab="/")

'种田/要/有/个明/白账/，/投本/要/赚/利润/是/起码/的/道理/。'